# imports

In [ ]:
import numpy as np
import pandas as pd
import os
import random

# This should print the project folder
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

In [ ]:
user_folder = <path to user folder>

In [2]:
import arcpy  
from arcpy.ia import * 
arcpy.CheckExtension("ImageAnalyst") 

'Available'

In [3]:
from arcpy.sa import *
arcpy.CheckExtension('Spatial')

'Available'

In [ ]:
arcpy.env.workspace = os.getcwd()
os.getcwd()

# Rename training/test samples

We start by assigning a number to each training sample

In [ ]:
import random
tile_sizes = [256,512]
pixel_sizes = [10,25]

for ts in tile_sizes:
    for ps in pixel_sizes:
        # Path to training samples folder
        ts_folder = f'data/training_samples/ts_{ts}_{ps}/ts_svalbard'

        # List all the folders in the ts folder
        folders = [f for f in os.listdir(ts_folder) if os.path.isdir(os.path.join(ts_folder, f))]

        # Rename each folder 
        for int, folder in enumerate(folders, start=1):
            old_folder_path = os.path.join(ts_folder, folder)
            new_folder_name = f"sample_{int}"
            new_folder_path = os.path.join(ts_folder, new_folder_name)
            
            # Rename the folder
            os.rename(old_folder_path, new_folder_path)
            print(f"Renamed '{folder}' to '{new_folder_name}'")

# Random test/train splits

The svalbard model has 100 samples. We first generate a random list of numbers

In [5]:
import random
# Seed for reproductibility
random.seed(1)

# Generate a random list of the numbers between 1 and 158
indices = random.sample(range(1, 159), 158)

print(indices)


[35, 146, 17, 66, 31, 127, 116, 121, 98, 54, 25, 125, 8, 100, 111, 1, 115, 69, 59, 27, 82, 157, 6, 7, 3, 150, 56, 109, 137, 57, 113, 151, 64, 71, 30, 45, 124, 87, 29, 133, 140, 38, 134, 149, 108, 72, 83, 13, 24, 81, 93, 117, 16, 96, 43, 114, 92, 65, 55, 101, 86, 148, 39, 37, 76, 126, 99, 51, 94, 5, 62, 32, 52, 142, 23, 47, 147, 48, 12, 129, 155, 14, 21, 67, 91, 131, 63, 4, 61, 136, 40, 74, 22, 78, 122, 128, 143, 50, 144, 158, 132, 36, 15, 26, 33, 84, 95, 53, 103, 18, 104, 79, 105, 88, 97, 85, 9, 34, 112, 77, 28, 138, 154, 110, 60, 73, 139, 106, 153, 80, 46, 120, 123, 49, 44, 20, 75, 11, 58, 145, 130, 141, 156, 107, 2, 41, 42, 89, 19, 102, 70, 135, 10, 119, 68, 118, 90, 152]


We will set aside 15 set out of 95 for testing purposes. This will remain untouched until the final model evaluation

In [6]:
test_split = []

for i in indices[128:]:
    test_split.append(f'/data/training_samples/ts_256_25/ts_svalbard/sample_{i}')

print(len(test_split))
print(test_split)

30
['/data/training_samples/ts_256_25/ts_svalbard/sample_153', '/data/training_samples/ts_256_25/ts_svalbard/sample_80', '/data/training_samples/ts_256_25/ts_svalbard/sample_46', '/data/training_samples/ts_256_25/ts_svalbard/sample_120', '/data/training_samples/ts_256_25/ts_svalbard/sample_123', '/data/training_samples/ts_256_25/ts_svalbard/sample_49', '/data/training_samples/ts_256_25/ts_svalbard/sample_44', '/data/training_samples/ts_256_25/ts_svalbard/sample_20', '/data/training_samples/ts_256_25/ts_svalbard/sample_75', '/data/training_samples/ts_256_25/ts_svalbard/sample_11', '/data/training_samples/ts_256_25/ts_svalbard/sample_58', '/data/training_samples/ts_256_25/ts_svalbard/sample_145', '/data/training_samples/ts_256_25/ts_svalbard/sample_130', '/data/training_samples/ts_256_25/ts_svalbard/sample_141', '/data/training_samples/ts_256_25/ts_svalbard/sample_156', '/data/training_samples/ts_256_25/ts_svalbard/sample_107', '/data/training_samples/ts_256_25/ts_svalbard/sample_2', '/d

In [7]:
train_split = []

for i in indices[:128]:
    train_split.append(f'/data/training_samples/ts_256_25/ts_svalbard/sample_{i}')

print(len(train_split))
print(train_split)

128
['/data/training_samples/ts_256_25/ts_svalbard/sample_35', '/data/training_samples/ts_256_25/ts_svalbard/sample_146', '/data/training_samples/ts_256_25/ts_svalbard/sample_17', '/data/training_samples/ts_256_25/ts_svalbard/sample_66', '/data/training_samples/ts_256_25/ts_svalbard/sample_31', '/data/training_samples/ts_256_25/ts_svalbard/sample_127', '/data/training_samples/ts_256_25/ts_svalbard/sample_116', '/data/training_samples/ts_256_25/ts_svalbard/sample_121', '/data/training_samples/ts_256_25/ts_svalbard/sample_98', '/data/training_samples/ts_256_25/ts_svalbard/sample_54', '/data/training_samples/ts_256_25/ts_svalbard/sample_25', '/data/training_samples/ts_256_25/ts_svalbard/sample_125', '/data/training_samples/ts_256_25/ts_svalbard/sample_8', '/data/training_samples/ts_256_25/ts_svalbard/sample_100', '/data/training_samples/ts_256_25/ts_svalbard/sample_111', '/data/training_samples/ts_256_25/ts_svalbard/sample_1', '/data/training_samples/ts_256_25/ts_svalbard/sample_115', '/d

# Batch model training

## batch pretrain

In [ ]:
arcpy.env.workspace = os.getcwd()
os.getcwd()


SAM

In [6]:
import random

tile_sizes = [256,512]
pixel_sizes = [10,25]

backbones = ['VIT_B','VIT_L']

for tilesize in tile_sizes:
    for pixelsize in pixel_sizes:
        # create train folder list
        train_list = []

        root = f"data/training_samples/ts_{tilesize}_{pixelsize}/ts_ns2019"
        for folder in os.listdir(root):
            folder_path = '/' + os.path.join(root,folder)
            train_list.append(folder_path)
            
        root = f"data/training_samples/ts_{tilesize}_{pixelsize}/ts_rune"
        for folder in os.listdir(root):
            folder_path = '/' + os.path.join(root,folder)
            train_list.append(folder_path)
        
        # shuffle to avoid validation bias
        random.shuffle(train_list)

        for backbone in backbones:
            if tilesize == 256:
                bs = 64
            if tilesize == 512:
                bs = 16
            if backbone == 'VIT_L':
                bs = bs / 4

            print(f'training for {tilesize}m x {pixelsize}m for {backbone} and batchsize = {bs}')
            with arcpy.EnvManager(processorType="GPU"):
                TrainDeepLearningModel(
                    in_folder = train_list,
                    out_folder = f'/models/pretrained/samlora_{tilesize}_{pixelsize}_{backbone}', 
                    max_epochs=100,
                    model_type="SAMLORA",
                    batch_size=bs,
                    arguments=f"class_balancing True;mixup False;focal_loss False;ignore_classes #;dice_loss_fraction 0.5",
                    learning_rate=None,
                    backbone_model=backbone,
                    validation_percentage=10,
                    stop_training="STOP_TRAINING",
                    freeze="UNFREEZE_MODEL",
                    augmentation="DEFAULT",
                    augmentation_parameters=None,
                    chip_size = tilesize,
                    resize_to="",
                    weight_init_scheme="",
                    monitor="VALID_LOSS"
                )

epoch,train_loss,valid_loss,accuracy,dice,time
0,0.517236,0.308145,0.996732,0.631851,25:19
1,0.500766,0.271038,0.997015,0.676395,25:18
2,0.513386,0.256521,0.997224,0.692954,25:17
3,0.501986,0.251423,0.997267,0.698160,25:19
4,0.503335,0.255145,0.997185,0.694378,25:18
5,0.478140,0.234944,0.997125,0.715768,25:17
6,0.506126,0.251309,0.997288,0.694808,25:17
7,0.490670,0.239064,0.997204,0.710893,25:18
8,0.478991,0.238973,0.997163,0.709628,25:18
9,0.480970,0.231742,0.997377,0.718660,25:20


Epoch 17: early stopping
Computing model metrics...


DEEPLAB

In [5]:
tile_sizes = [512]
pixel_sizes = [25]

backbones = ['RESNET50','RESNET101']

for tilesize in tile_sizes:
    for pixelsize in pixel_sizes:
        # create train folder list
        train_list = []

        root = f"data/training_samples/ts_{tilesize}_{pixelsize}/ts_ns2019"
        for folder in os.listdir(root):
            folder_path = '/' + os.path.join(root,folder)
            train_list.append(folder_path)
            
        root = f"data/training_samples/ts_{tilesize}_{pixelsize}/ts_rune"
        for folder in os.listdir(root):
            folder_path = '/' + os.path.join(root,folder)
            train_list.append(folder_path)

        random.shuffle(train_list)

        for backbone in backbones: 
            if tilesize == 256:
                bs = 64
            if tilesize == 512:
                bs = 16
            if backbone == 'RESNET101':
                bs =  bs / 2
            print(f'training for {tilesize}m x {pixelsize}m for {backbone} and batchsize = {bs}')
            with arcpy.EnvManager(processorType="GPU"):
                arcpy.ia.TrainDeepLearningModel(
                    in_folder = train_list,
                    out_folder = f"/models/pretrained/deeplab_{tilesize}_{pixelsize}_{backbone}",  
                    max_epochs=100,
                    model_type="DEEPLAB", 
                    arguments=f"class_balancing True;mixup False;focal_loss False;ignore_classes #;dice_loss_fraction 0.5",
                    batch_size=bs,
                    learning_rate=None,
                    backbone_model=backbone,
                    validation_percentage=10,
                    stop_training="STOP_TRAINING",
                    freeze="UNFREEZE_MODEL",
                    augmentation="DEFAULT",
                    augmentation_parameters=None,
                    chip_size=tilesize,
                    monitor="VALID_LOSS"
                )

Epoch 12: early stopping
Computing model metrics...


In [ ]:
break

Initiate random hyperparameters. as recommended by Goodfellow et al 2015 and Djikstra & Bengio 2012

In [ ]:
# # Random search
# random.seed(1)

# # Define search space for each hyperparameter
# backbones = ['RESNET50', 'RESNET101']
# # bsizes = [4, 8, 12, 16, 22]
# # lrates = [1e-5, 1e-4, 1e-3]

# # Number of random combinations to test 
# num_combinations = 8

# # Create a set to track unique combinations
# unique_combinations = set()

# # Randomly sample combinations ensuring no duplicates
# while len(unique_combinations) < num_combinations:
#     backbone = random.choice(backbones)
#     batch_size = random.choice(bsizes)
#     learning_rate = random.choice(lrates)
    
#     # Use a tuple to store the combination as a hashable element in the set
#     combination = (backbone, batch_size, learning_rate)
    
#     # Add the combination to the set (duplicates will be ignored)
#     unique_combinations.add(combination)

# # Convert the set of combinations back into a list for further processing
# random_combinations = [{'backbone': b, 'batch_size': s, 'learning_rate': lr} for b, s, lr in unique_combinations]

# # Print the random combinations
# for i, config in enumerate(random_combinations):
#     print(f"Random search config {i+1}: {config}")

## Base models batch

In [7]:
arcpy.ResetEnvironments()

In [8]:
tile_sizes = [256,512]
pixel_sizes = [10,25]

N_train_sets = [2,8,32,128] # N = 4
b = 1

Define number of model per tilesize-pixelsize-backbone combination
    # if N = 7: 
        # if b = 1: 8 models per N. 8*7 = 56 models for deeplab. 112 base models. With transfer learning = Total of 224 models
        # if b = 2: 16 models per N. 16*7 = 112 models for Deeplab. 224 base models. With transfer learning = Total of 448 models

    # if N = 4: 
        # if b = 1: 8 models per N. 8*4 = 32 models for deeplab. 64 base models. With transfer learning = Total of 128 models
        # if b = 2: 16 models per N. 16*4 = 64 models for Deeplab. 128 base models. With transfer learning = Total of 256 models

### DeepLab

In [14]:
# make sure the workspace refers to the project folder
arcpy.env.workspace = os.getcwd()

# arcpy.env.processorType = 'GPU'

backbones = ['RESNET50', 'RESNET101']

for tilesize in tile_sizes:
    for pixelsize in pixel_sizes:
        # Load train split for tile format
        train_split = []
        for i in indices[:128]:
            train_split.append(f'/data/training_samples/ts_{tilesize}_{pixelsize}/ts_svalbard/sample_{i}')

        for N in N_train_sets: # for N in the set of training samples
            # Adaptive batchsize based on GPU memory
            for backbone in backbones:
                if backbone == 'RESNET50':
                    if tilesize == 256:
                        bs = 64
                    else: # if ts = 512
                        bs = 16
                else:
                    if tilesize == 256:
                        bs = 32
                    else: # if ts = 512
                        bs = 8
            
                if N <= 8:
                    bs = N
                
                for s in range(b): # for b, the number of bootstraps
                    if N <= 8:
                        N += 1 # 10% val fix for small numbers

                    # load N bootstrapped images from train split
                    train_in = random.sample(train_split,N)

                    if N <= 9:
                        N -= 1 #return N to initial value
                
                    out_path = f'/models/base/deeplab_{tilesize}_{pixelsize}_n{N}_{backbone}'
                    if not os.path.isdir(out_path.removeprefix('/')):
                        # train model
                        print(f'training model for ts {tilesize}, ps {pixelsize}, backbone {backbone}, with {N} images and batch size {bs}')
                        # print(f'input training data: {train_in}')
                        with arcpy.EnvManager(processorType="GPU"):
                            TrainDeepLearningModel(
                                in_folder = train_in,
                                out_folder = out_path,
                                max_epochs=100,
                                model_type="DEEPLAB",
                                batch_size=bs,
                                arguments="class_balancing True;mixup False;focal_loss False;ignore_classes #;dice_loss_fraction 0.5", # weighted combo loss function
                                learning_rate=None, # automatic optimization from an estimated learning curve
                                backbone_model=backbone,
                                pretrained_model=None,
                                validation_percentage=10,
                                stop_training="STOP_TRAINING",
                                freeze="UNFREEZE_MODEL",
                                augmentation="DEFAULT",
                                augmentation_parameters=None,
                                chip_size=tilesize,
                                resize_to="",
                                weight_init_scheme="", 
                                monitor="VALID_LOSS"
                            )
                    else:
                        print(f'{out_path} already exists, skipping')
    

epoch,train_loss,valid_loss,accuracy,dice,time
0,0.626564,0.535755,0.970991,0.312233,01:15
1,0.516920,0.402992,0.986600,0.511662,01:16
2,0.457923,0.361409,0.985277,0.496569,01:16
3,0.412021,0.290429,0.988355,0.543668,01:16
4,0.386397,0.252109,0.991329,0.587188,01:15
5,0.364923,0.255169,0.988469,0.555338,01:16
6,0.363651,0.289812,0.988038,0.508385,01:16
7,0.364088,0.227577,0.992402,0.595130,01:15
8,0.360778,0.282222,0.988796,0.496962,01:16
9,0.357285,0.250150,0.986843,0.564794,01:16


Epoch 17: early stopping
Computing model metrics...


### SAMLoRA

In [15]:
# make sure the workspace refers to the project folder
arcpy.env.workspace = os.getcwd()

# arcpy.env.processorType = 'GPU'

backbones = ['VIT_B','VIT_L']

for tilesize in tile_sizes:
    for pixelsize in pixel_sizes:
        # Load train split for tile format
        train_split = []
        for i in indices[:128]:
            train_split.append(f'/data/training_samples/ts_{tilesize}_{pixelsize}/ts_svalbard/sample_{i}')
        for N in N_train_sets: # for N in the set of training samples
            # Adaptive batchsize based on GPU memory
            for backbone in backbones:
                if tilesize == 256:
                    if backbone == 'VIT_B':
                        bs = 64
                    else: # if bb = Vit-L
                        bs = 16
                else: # if ts = 512
                    if backbone == 'VIT_B':
                        bs = 16
                    else: # bb = Vit-L
                        bs = 4
            
                if (N <= 8) and (bs != 4):
                    bs = N

                for s in range(b): # for b, the number of bootstraps
                    if N <= 8:
                        N += 1 # 10% val fix for small numbers

                    # load N bootstrapped images from train split
                    train_in = random.sample(train_split,N)

                    if N <= 9:
                        N -= 1 #return N to initial value

                    out_path = f'/models/base/samlora_{tilesize}_{pixelsize}_n{N}_{backbone}'
                    if not os.path.isdir(out_path.removeprefix('/')):
                        # train model
                        print(f'training for ts {tilesize}, ps {pixelsize}, backbone {backbone}, with {N} images and batch size {bs}...')
                        # print(f'input training data: {train_in}')
                        with arcpy.EnvManager(processorType="GPU"):
                            TrainDeepLearningModel(
                                in_folder = train_in,
                                out_folder = out_path,
                                max_epochs=100,
                                model_type="SAMLORA",
                                batch_size=bs,
                                arguments="class_balancing True;mixup False;focal_loss False;ignore_classes #;dice_loss_fraction 0.5", # weighted combo loss function
                                learning_rate=None, # automatic optimization from an estimated learning curve
                                backbone_model=backbone,
                                pretrained_model=None,
                                # pretrained_model= None,
                                validation_percentage=10,
                                stop_training="STOP_TRAINING",
                                freeze="UNFREEZE_MODEL",
                                augmentation="DEFAULT",
                                augmentation_parameters=None,
                                chip_size=tilesize,
                                resize_to="",
                                weight_init_scheme="", 
                                monitor="VALID_LOSS"
                            )
                    else: 
                        print(f'{out_path} already exists')

epoch,train_loss,valid_loss,accuracy,dice,time
0,0.562685,0.289270,0.992082,0.661609,03:29
1,0.520248,0.264214,0.993252,0.691240,03:31
2,0.492761,0.232817,0.992999,0.731147,03:13
3,0.466531,0.243246,0.994088,0.712817,03:12
4,0.484075,0.228792,0.993948,0.733622,03:28
5,0.484205,0.234356,0.992661,0.725506,03:28
6,0.470976,0.235827,0.993149,0.720656,03:32
7,0.481245,0.225926,0.993603,0.732241,03:30
8,0.485167,0.260763,0.991693,0.694133,03:30
9,0.483335,0.261862,0.992806,0.689228,03:29


Epoch 16: early stopping
Computing model metrics...


## TL models batch

In [16]:
arcpy.ResetEnvironments()

Define number of model per tilesize-pixelsize-backbone combination
    # if N = 7: 
        # if b = 1: 8 models per N. 8*7 = 56 models for deeplab. 112 base models. With transfer learning = Total of 224 models
        # if b = 2: 16 models per N. 16*7 = 112 models for Deeplab. 224 base models. With transfer learning = Total of 448 models

    # if N = 4: 
        # if b = 1: 8 models per N. 8*4 = 32 models for deeplab. 64 base models. With transfer learning = Total of 128 models
        # if b = 2: 16 models per N. 16*4 = 64 models for Deeplab. 128 base models. With transfer learning = Total of 256 models

In [8]:
tile_sizes = [256,512]
pixel_sizes = [10,25]

N_train_sets = [2,4,8] # N = 4
b = 1

### DeepLab

In [18]:
len(train_split)

128

### SAMLoRA

In [7]:
tile_sizes = [256,512]
pixel_sizes = [10,25]

N_train_sets = [2,8,32,128] # N = 4
b = 1

In [10]:
# make sure the workspace refers to the project folder
arcpy.env.workspace = os.getcwd()

# arcpy.env.processorType = 'GPU'

backbones = ['VIT_B','VIT_L']

for tilesize in tile_sizes:
    for pixelsize in pixel_sizes:
        # Load train split for tile format
        train_split = []
        for i in indices[:128]:
            train_split.append(f'/data/training_samples/ts_{tilesize}_{pixelsize}/ts_svalbard/sample_{i}')
        # list of number of training samples
        for N in N_train_sets: # for N in the set of training samples
            # if N <= 16: 
            #     b = 2
            # else: 
            #     b = 2
            for backbone in backbones:
                if backbone == 'VIT_B':
                    if tilesize == 256:
                        bs = 64
                    else: # if ts = 512
                        bs = 16
                else: # if VIT_L
                    if tilesize == 256:
                        bs = 16
                    else: # if ts = 512
                        bs = 4
            
                if (N <= 8) and (bs != 4):
                    bs = N

                for s in range(b): # for b, the number of bootstraps
                    if N <= 8:
                        N += 1 # 10% val fix for small numbers

                    # load N bootstrapped images from train split
                    train_in = random.sample(train_split,N)

                    if N <= 9:
                        N -= 1 #return N to initial value

                    out_path = f'/models/transfer_learning/samlora_{tilesize}_{pixelsize}_n{N}_{backbone}'
                    if not os.path.isdir(out_path.removeprefix('/')):
                    # train model
                        print(f'training for ts {tilesize}, ps {pixelsize}, backbone {backbone}, with {N} images and batch size {bs}...')
                        # print(f'input training data: {train_in}')
                        with arcpy.EnvManager(processorType="GPU"):
                            TrainDeepLearningModel(
                                in_folder = train_in,
                                out_folder = out_path,
                                max_epochs=100,
                                model_type="SAMLORA",
                                batch_size=bs,
                                arguments="class_balancing True;mixup False;focal_loss False;ignore_classes #;dice_loss_fraction 0.5", # weighted combo loss function
                                learning_rate=None, # automatic optimization from an estimated learning curve
                                backbone_model=backbone,
                                pretrained_model=f'/models/pretrained/samlora_{tilesize}_{pixelsize}_{backbone}/samlora_{tilesize}_{pixelsize}_{backbone}.emd',
                                # pretrained_model= None,
                                validation_percentage=10,
                                stop_training="STOP_TRAINING",
                                freeze="UNFREEZE_MODEL",
                                augmentation="DEFAULT",
                                augmentation_parameters=None,
                                chip_size=tilesize,
                                resize_to="",
                                weight_init_scheme="", 
                                monitor="VALID_LOSS"
                            )
                    else:
                        print(f"{out_path.removeprefix('/')} already exists")
    

epoch,train_loss,valid_loss,accuracy,dice,time
0,0.650785,0.332560,0.998333,0.591970,00:02
1,0.552470,0.329494,0.998371,0.597549,00:02
2,0.518776,0.326765,0.998394,0.600948,00:02
3,0.514049,0.323361,0.998444,0.607692,00:02
4,0.501969,0.319129,0.998493,0.614634,00:02
5,0.507002,0.314168,0.998539,0.621915,00:02
6,0.492409,0.307926,0.998573,0.626747,00:02
7,0.460815,0.299937,0.998665,0.642127,00:04
8,0.474219,0.290476,0.998741,0.652632,00:02
9,0.475676,0.280865,0.998817,0.663043,00:02


Epoch 19: early stopping
Computing model metrics...
models/transfer_learning/samlora_512_25_n8_VIT_B already exists
models/transfer_learning/samlora_512_25_n8_VIT_L already exists
models/transfer_learning/samlora_512_25_n32_VIT_B already exists
models/transfer_learning/samlora_512_25_n32_VIT_L already exists
models/transfer_learning/samlora_512_25_n128_VIT_B already exists
models/transfer_learning/samlora_512_25_n128_VIT_L already exists


# Normalize training samples(0-1) 

In [ ]:
break

In [7]:
# arcpy.env.overwriteOutput = True
# import rasterio

# # Path to the root folder containing subfolders
# root_folder = rf"C:\Users\{user_folder}\.1\--Data\training_samples\ts_512_10\ts_ns2019"

# # Iterate through the subfolders in the root folder
# for folder in os.listdir(root_folder):
#     folder_path = os.path.join(root_folder, folder)

#     # define the images folder path
#     images_folder_path = os.path.join(folder_path, 'images')
#     normalized_images_path = os.makedirs(os.path.join(folder_path, 'images_norm'))
    
#     # iterate through all files in 'images'
#     for file_name in os.listdir(images_folder_path):
#         file_path = os.path.join(images_folder_path, file_name)

#         # open the image with rasterio
#         if file_name.endswith('.tif'):
#             with rasterio.open(file_path) as src:
#                 image = src.read(1)  # single band raster
                
#                 # define the min max raster values with rasterio ## I found this is faster than calculating raster statistics with arcpy
#                 min = np.min(image)
#                 max = np.max(image)
                
#             # raster calculator, min-max normalize
#             normalized = RasterCalculator(
#                 rasters = [file_path], 
#                 input_names = 'x', 
#                 expression = f'(x - {min}) / ({max} - {min})'
#                 )

#             # arcpy save raster (cannot overwrite) 
#             out_path = f'{folder_path}/images_norm/{file_name}'
#             normalized.save(out_path)
#             print(f'{file_name} normalized and saved at {out_path}')